In [19]:
import tabula


pdf_file = "PDF/d_pos_v6.pdf"
#pdf_file = "PDF/d_neg_v6.pdf"
dfs = tabula.read_pdf(pdf_file, pages="all", stream=True, encoding='cp1252')
tabula.environment_info()
print(len(dfs))

Python version:
    3.13.3 (v3.13.3:6280bb54784, Apr  8 2025, 10:47:54) [Clang 15.0.0 (clang-1500.3.9.4)]
Java version:
    openjdk version "20.0.2" 2023-07-18
OpenJDK Runtime Environment (build 20.0.2+9-78)
OpenJDK 64-Bit Server VM (build 20.0.2+9-78, mixed mode, sharing)
tabula-py version: 2.10.0
platform: macOS-15.5-x86_64-i386-64bit-Mach-O
uname:
    uname_result(system='Darwin', node='Freya.local', release='24.5.0', version='Darwin Kernel Version 24.5.0: Tue Apr 22 19:53:26 PDT 2025; root:xnu-11417.121.6~2/RELEASE_X86_64', machine='x86_64')
linux_distribution: ('Darwin', '24.5.0', '')
mac_ver: ('15.5', ('', '', ''), 'x86_64')
3


In [20]:
print(len(dfs))

3


In [21]:
dfs[0]

,N ame:,Jane Doe,Name:,Dr Requesta,Laborat o ry:,Gentest UK
0,Date of birth:,18 March 1970,Organisation:,Chester Hospital,Telephone:,01223 666555
1,Sex:,Female,Telephone:,01223 555555,Date receive d:,26 February 2018
2,NHS number:,NH00198,Copies to:,Dr A. Nother,Date reported :,12 March 2018
3,Sample type:,Blood,NaN,Dr X. Tra,Authorised by:,ATester


In [22]:
dfs[1]

,N ame:,Jane Doe,Name:,Dr Requesta,Laborat o ry:,Gentest UK
0,Date of birth:,18 March 1970,Organisation:,Chester Hospital,Telephone:,01223 666555
1,Sex:,Female,Telephone:,01223 555555,Date receive d:,26 February 2018
2,NHS number:,NH00198,Copies to:,Dr A. Nother,Date reported :,12 March 2018
3,Sample type:,Blood,NaN,Dr X. Tra,Authorised by:,ATester


In [23]:
dfs[2]

,Gene,Zygosity,Inheritance,HGVS Description,Location: GRCh38 (hg38),*Classification
0,BRCA1,Heterozygous,Autosomal,c.3143delG p.(Gly1048ValfsTer14),17: 43092388 (GRCh38),Pathogenic
1,NaN,NaN,dominant,NaN,NaN,NaN
2,BRCA2,No clinically,NaN,NaN,NaN,Not
3,NaN,pathogenic,NaN,NaN,NaN,applicable
4,NaN,variant detected,NaN,NaN,NaN,NaN
5,PALB2,No clinically,NaN,NaN,NaN,Not
6,NaN,pathogenic,NaN,NaN,NaN,applicable
7,NaN,variant detected,NaN,NaN,NaN,NaN


Based on https://build.fhir.org/ig/HL7/genomics-reporting/

I think we can build a variant http://hl7.org/fhir/uv/genomics-reporting/StructureDefinition/variant

In [24]:
from fhir.resources.observation import Observation,ObservationComponent
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding

df = dfs[len(dfs)-1]
df['Gene'] = df['Gene'].astype(str)
df['HGVS Description'] = df['HGVS Description'].astype(str)
df.dtypes

Gene                       object
Zygosity                   object
Inheritance                object
HGVS Description           object
Location: GRCh38 (hg38)    object
*Classification            object
dtype: object

In [25]:


for i in range(0, len(df)):
    if df.loc[i,'Gene'] is not None and df.loc[i,'Gene'] != 'nan':
        print('\n',df.loc[i,'Gene'],'\n')
        observationBase = {
            "status" : "final",
            "category":[
                {
                    "coding" : [
                        {
                            "system" : "http://terminology.hl7.org/CodeSystem/observation-category",
                            "code" : "laboratory"
                        }
                    ]
                },
                {
                    "coding" : [
                        {
                            "system" : "http://terminology.hl7.org/CodeSystem/v2-0074",
                            "code" : "GE"
                        }
                    ]
                }
            ],
            "code" : {
                "coding" : [
                    {
                        "system" : "http://loinc.org",
                        "code" : "69548-6",
                        "display" : "Genetic variant assessment"
                    }
                ]
            },
            "valueCodeableConcept" : {
                "coding" : [
                    {
                        "system" : "http://loinc.org",
                        "code" : "LA9633-4",
                        "display" : "Present"
                    }
                ]
            },
            "subject" : {
                "reference" : "urn:uuid:d6faafcf-db64-4c11-9da8-25f36774c1bd",
                "identifier" : {
                    "system" : "https://fhir.nhs.uk/Id/nhs-number",
                    "value" : "9449305552"
                },
                "display" : "Octavia CHISLETT"
            },
            "method" : {
                "coding" : [
                    {
                        "system" : "http://loinc.org",
                        "code" : "LA26398-0",
                        "display" : "Sequencing"
                    }
                ]
            },
            "component" : []
        }
        obs = Observation(**observationBase)
        genename = "NK"
        if df.loc[i,'Gene'] == 'BRCA1':
            genename = "HGNC:1100"
        if df.loc[i,'Gene'] == 'BRCA2':
            genename = "HGNC:1101"
        if df.loc[i,'Gene'] == 'PALB2':
            genename = "HGNC:26144"
        geneData = {
            "code" : {
                "coding" : [
                    {
                        "system" : "http://loinc.org",
                        "code" : "48018-6"
                    }
                ]
            },
            "valueCodeableConcept" : {
                "coding" : [
                    {
                        "system" : "http://www.genenames.org",
                        "code" : genename,
                        "display" : df.loc[i,'Gene']
                    }
                ]
            }
        }
        gene = ObservationComponent(**geneData)
        obs.component.append(gene)

        if df.loc[i,'Zygosity'] == 'Heterozygous':
            alleData = {
                "code" : {
                    "coding" : [
                        {
                            "system" : "http://loinc.org",
                            "code" : "53034-5",
                            "display" : "Allelic state"
                        }
                    ]
                },
                "valueCodeableConcept" : {
                    "coding" : [
                        {
                            "system" : "http://loinc.org",
                            "code" : "LA6706-1",
                            "display" : "Heterozygous"
                        }
                    ]
                }
            }
            alle = ObservationComponent(**alleData)
            obs.component.append(alle)
        if df.loc[i,'HGVS Description'] is not None and df.loc[i,'HGVS Description'] != 'nan':
            code = df.loc[i,'Gene'] + ":" + df.loc[i,'HGVS Description']
            hgvsData = {
                "code" : {
                    "coding" : [
                        {
                            "system" : "http://loinc.org",
                            "code" : "48004-6",
                            "display": "DNA change"
                        }
                    ]
                },
                "valueCodeableConcept" : {
                    "coding" : [
                        {
                            "system" : "http://varnomen.hgvs.org",
                            "code" : code
                        }
                    ]
                }
            }
            hgvs = ObservationComponent(**hgvsData)
            obs.component.append(hgvs)

        json_str = obs.model_dump_json()
        print(json_str)


 BRCA1 

{"resourceType":"Observation","status":"final","category":[{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/observation-category","code":"laboratory"}]},{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v2-0074","code":"GE"}]}],"code":{"coding":[{"system":"http://loinc.org","code":"69548-6","display":"Genetic variant assessment"}]},"subject":{"reference":"urn:uuid:d6faafcf-db64-4c11-9da8-25f36774c1bd","identifier":{"system":"https://fhir.nhs.uk/Id/nhs-number","value":"9449305552"},"display":"Octavia CHISLETT"},"valueCodeableConcept":{"coding":[{"system":"http://loinc.org","code":"LA9633-4","display":"Present"}]},"method":{"coding":[{"system":"http://loinc.org","code":"LA26398-0","display":"Sequencing"}]},"component":[{"code":{"coding":[{"system":"http://loinc.org","code":"48018-6"}]},"valueCodeableConcept":{"coding":[{"system":"http://www.genenames.org","code":"HGNC:1100","display":"BRCA1"}]}},{"code":{"coding":[{"system":"http://loinc.org","code":"53034-